### Task: sorting digits (e.g. [3,1,2] -> [1,2,3])

* Encoder-Decoder (Sutskever et al. 2014)

In [1]:
# Add custom import path

import sys
sys.path.insert(0, '/home/jacobsuwang/Documents/UTA2018/NEURAL-NETS/ATTENTION/CODE/01-import-folder')

### MAKING DATA

In [225]:
import utils
import random
import numpy as np

VOCAB = set(['PAD','EOS','1','2','3','4','5','6','7','8','9','0'])
NUMBERS = ['1','2','3','4','5','6','7','8','9','0']
MAX_LEN = len(NUMBERS) + 2
# WORD2IDX = {'PAD':0,'EOS':1,'1':2,'2':3,'3':4,'4':5,'5':6,'6':7,'7':8,'8':9,'9':10,'0':11}
WORD2IDX = {'PAD':0,'EOS':1,'1':2,'2':3,'3':4,'4':5,'5':6,'6':7,'7':8,'8':9,'9':10,'0':11}
IDX2WORD = {idx:word for word,idx in WORD2IDX.iteritems()}
BATCH_SIZE = 10


def random_datum(n):
    input_seq = list(np.random.choice(NUMBERS, n, replace=False)) 
        # e.g. ['5', '6', '3', '9', '1'].
    sorted_seq = sorted(input_seq)
    output_seq = [input_seq.index(word)+2 for word in sorted_seq]
        # index in ascending.
        # e.g. [4, 2, 0, 1, 3], for the input above.
    input_seq = input_seq
    return input_seq, output_seq

def encode_seq(seq):
    return [WORD2IDX[word] for word in seq]

def decode_seq(seq):
    return [IDX2WORD[idx] for idx in seq]

def decode_order(seq):
    return [aug_idx-2 for aug_idx in seq]

def decode_by_index(seq, idx_seq, end_idx, correction=-2): # -2: PAD and EOS
    decoded = []
    for idx in range(end_idx):
        decoded.append(seq[idx_seq[idx]+correction])
    return decoded

def random_batch(batch_size, input_length_from=2, input_length_to=MAX_LEN-2):
    if input_length_from >= input_length_to:
        raise ValueError('length_from >= length_to')
    input_lengths = np.random.randint(input_length_from, input_length_to, size=batch_size)
    input_batch, output_batch = [], []
    for length in input_lengths:
        input_seq, output_seq = random_datum(length)
        input_batch.append(encode_seq(input_seq))
        output_batch.append(output_seq)
    return input_batch, output_batch    


In [170]:
a,b = random_datum(10)
print a
print decode_order(b)

['5', '3', '0', '8', '6', '1', '2', '9', '7', '4']
[2, 5, 6, 1, 9, 0, 4, 8, 3, 7]


In [5]:
a, b = random_batch(batch_size=2)
print a
print
print b

[[8, 4], [11, 2, 6, 9, 3]]

[[1, 0], [0, 1, 4, 2, 3]]


In [6]:
decode_seq(a[0])

['7', '3']

In [7]:
utils.batch(a, max_sequence_length=MAX_LEN, custom_pad=MAX_LEN-1)

(array([[ 8, 11],
        [ 4,  2],
        [11,  6],
        [11,  9],
        [11,  3],
        [11, 11],
        [11, 11],
        [11, 11],
        [11, 11],
        [11, 11],
        [11, 11],
        [11, 11]], dtype=int32), [2, 5])

### MAKING MODEL

In [9]:
import numpy as np
import tensorflow as tf

from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple

In [212]:
# Graph

tf.reset_default_graph()
sess = tf.InteractiveSession()

vocab_size = len(VOCAB)
input_embedding_size = 20

encoder_hidden_units = 20
decoder_hidden_units = encoder_hidden_units * 2 # because encoder is going to be bidirectional.

#                    decoder 
#                    target
# 
# [] -> [] -> [#] -> [] -> []
#                     |    ^
# encoder             |____|
# inputs             

encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs') # [max_time, batch_size]
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length') 
    # this takes a vector (length=batch_size), where each cell is the length of the
    # correponding data entry. this doesn't affect time_major op.
decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')

embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs) # [max_time, batch_size, emb_size]

encoder_cell = LSTMCell(encoder_hidden_units)
((encoder_fw_outputs,encoder_bw_outputs), # both have [max_time, batch_size, emb_size]
 (encoder_fw_final_state,encoder_bw_final_state)) = ( # state tuples: (c=[batch_size,emb_size],h=same)
        tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                        cell_bw=encoder_cell,
                                        inputs=encoder_inputs_embedded,
                                        sequence_length=encoder_inputs_length,
                                        dtype=tf.float32, time_major=True)
    )

# concat fw-bw separately, then make a combined final state!
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2) # concat on emb dim.
encoder_final_state_c = tf.concat((encoder_fw_final_state.c, encoder_bw_final_state.c), 1) # same thing.
encoder_final_state_h = tf.concat((encoder_fw_final_state.h, encoder_bw_final_state.h), 1)
encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h
)

decoder_cell = LSTMCell(decoder_hidden_units)
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))
    # getting the shape of a tensor [max_time, batch_size].
    # doc: Unpacks the given dimension of a rank-`R` tensor into rank-`(R-1)` tensors.
    # WHY: ?dynamically keeping track of the shape?
decoder_lengths = encoder_inputs_length + 3 # +2 steps, +1 for EOS.

W = tf.Variable(tf.random_uniform([decoder_hidden_units, vocab_size], -1, 1), dtype=tf.float32) # for dec only!
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)
    # shared weights in the dynamic unrolling of the decoder.
    # W shape: [emb_concat, vocab]
    # it will be matmuled in output * W: [batch_size, emb_concat] * [emb_concat, vocab]
    # get: [batch_size, vocab], where we have allthe predictions (as multinomials)

# prepare tokens for each time step
eos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='EOS') # [batch_size]
pad_time_slice = tf.zeros([batch_size], dtype=tf.int32, name='PAD')
eos_step_embedded = tf.nn.embedding_lookup(embeddings, eos_time_slice) # [max_time, batch_size]
pad_step_embedded = tf.nn.embedding_lookup(embeddings, pad_time_slice)

# Loop feed (doc: tf.nn.raw_rnn?)
# (time, prev_cell_output, prev_cell_state, prev_loop_state) ->
# (elem_finished, input, cell_state, output, loop_state)

# handles first state (i.e. corresponds to the last state of the encoder)
#
#     state feed only (enc final state)
#         |
#         v
#      # --> #
#   last     first 
#   of enc   of dec
#            ^
#            |
#           EOS
#
def loop_fn_initial():
    initial_elements_finished = (0 >= decoder_lengths) # all false (i.e. not done) at the init step.
    initial_input = eos_step_embedded                  # it's a [batch_size] length boolean vector.
        # "input": it's the input for the next state.
        # in this case, the first cell of the decoder.
    initial_cell_state = encoder_final_state
    initial_cell_output = None # these two None help us
    initial_loop_state = None  # checking whether we are at the init step.
    return (initial_elements_finished,
            initial_input,
            initial_cell_state,
            initial_cell_output,
            initial_loop_state)

# handles the transitions in decoder after the first state
#             ___
#  output ->  |  |
#             # -|-> #
#              / |   ^
#         state  |___| <- next_input (inpt)
#
def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):
    def get_next_input():
        # at the first cell of the decoder, we take the feed from 
        # the final state of the encoder (handled by loop_fn_init),
        # feed = EOS embedding
        # and compute the first prediction. 
        output_logits = tf.add(tf.matmul(previous_output, W), b)
            # output * W: [batch_size, emb_concat] * [emb_concat, vocab]
            # get: [batch_size, vocab], where we have all the predictions (as multinomials)
        prediction = tf.argmax(output_logits, axis=1)
        next_input = tf.nn.embedding_lookup(embeddings, prediction)
        return next_input
    elements_finished = (time >= decoder_lengths) # again a [batch_size] boolean vector.
        # this returns a boolean tensor, e.g. [1, 1, 1, 0]
        # this means the first three steps are done, but not the last.
        # when all the steps are done, i.e. time (the real time) is larger than
        # the specified max decoding steps, the vector is all 1.
        # then the next line will return 1.
    finished = tf.reduce_all(elements_finished) # maps to boolean scalar.
    inpt = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
        # if finished, return a pad for next input (i.e. the feed to next step)
        # otherwise, return get_next_input as usual.
    state = previous_state
    output = previous_output
    loop_state = None
    # outputs:
    # elements_finished: a [batch_size] boolean vector.
    # inpt: [batch_size, emb_size] tensor for the next cell.
    # state: (c,h) tuole, raw_rnn takes care of it.
    # output: stored [batch_size, emb_size] tensor.
    # loop_state: rnn_raw takes care of it.
    return (elements_finished,
            inpt, 
            state,
            output,
            loop_state)

# combine the two fns above for a single looping function.
def loop_fn(time, previous_output, previous_state, previous_loop_state):
    # time: an int32 scalar raw_rnn uses to keep track of time-steps internally.
    # previous_output: [max_time, batch_size, emb_size] tensor.
    # previous_state: (c,h) tuple.
    # previous_loop_state: raw_rnn uses to keep track of where it is in the loop (automatic).
    if previous_state is None: # time = 0
        assert previous_output is None and previous_state is None
        return loop_fn_initial()
    else:
        return loop_fn_transition(time, previous_output, previous_state, previous_loop_state)

decoder_outputs_ta, decoder_final_state, _ = tf.nn.raw_rnn(decoder_cell, loop_fn) # we have an LSTM cell.
    # *_ta: the RNN output (TensorArray <- for dynamic use)
    # *_final_state: 2-tuple of [batch_size, emb_size] (i.e. c and h). of no use for seq2seq.
    # _: final_loop_state, which no one gives a fuck (used internally by *.raw_rnn backend).
decoder_outputs = decoder_outputs_ta.stack() # [max_time, batch_size, emb_concat]

decoder_max_step, decoder_batch_size, decoder_dim = tf.unstack(tf.shape(decoder_outputs))
decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_dim))
    # for matmul, we do
    # [max_time, batch_size, emb_concat], [max_time*batch_size, emb_concat]
decoder_logits_flat = tf.add(tf.matmul(decoder_outputs_flat, W), b)
decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_step, decoder_batch_size, vocab_size))
    # put it back into the original shaping scheme.
decoder_prediction = tf.cast(tf.argmax(decoder_logits, 2), dtype=tf.int32) # [max_time, batch_size]

# [SU] report accuracy here
correct_raw = tf.cast(tf.equal(decoder_prediction, decoder_targets), tf.int32)
mask = tf.cast(tf.not_equal(decoder_targets, WORD2IDX['PAD']), tf.int32) # EOSs are not 0ed out, there are BATCH_SIZE of them.
total_seqlen = tf.cast(tf.reduce_sum(encoder_inputs_length), tf.float32)
correct = tf.multiply(correct_raw, mask)
accuracy = tf.cast(tf.reduce_sum(correct)-BATCH_SIZE, tf.float32) / total_seqlen # -BATCH_SIZE correction
# accuracy = tf.cast(tf.reduce_sum(correct), tf.float32) / total_seqlen # w/o the correction.

# Optimization
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits
)
loss = tf.reduce_mean(stepwise_cross_entropy)
train_op = tf.train.AdamOptimizer().minimize(loss)

init = tf.global_variables_initializer()
sess.run(init)

In [224]:
def next_feed(batch_size):
    batch_enc, batch_dec = random_batch(batch_size)
    encoder_inputs_, encoder_inputs_lengths_ = utils.batch(batch_enc)
    decoder_targets_, _ = utils.batch([seq + [WORD2IDX['EOS']] + [WORD2IDX['PAD']]*2 for seq in batch_dec])
    return {
        encoder_inputs: encoder_inputs_,
        encoder_inputs_length: encoder_inputs_lengths_,
        decoder_targets: decoder_targets_
    }

loss_track = []

max_batches = 3001
batches_in_epoch = 1000
num_test_batches = 100

try:
    for batch in range(max_batches):
        fd = next_feed(BATCH_SIZE)
        _, l = sess.run([train_op, loss], fd)
        loss_track.append(l)
        if batch == 0 or batch % batches_in_epoch == 0:
            print('Batch {}'.format(batch))
            print('  minibatch loss: {} | accuracy {}'.format(*sess.run([loss, accuracy], fd)))
            # predict_ = sess.run(decoder_prediction, fd)
            predict_, lengths_ = sess.run([decoder_prediction, encoder_inputs_length], fd) # make use of seqlen

#             TEST BLOC
#
#             print fd[encoder_inputs].shape
#             print predict_.shape
#             print fd[decoder_targets].shape
#             print
#
#             print 'seqlen:'
#             print fd[encoder_inputs_length]
#             print 'prediction:'
#             print predict_
#             print 'true:'
#             print fd[decoder_targets]
#             print 'mask:'
#             print sess.run(mask, fd)
#             print 'correct-raw:'
#             print sess.run(correct_raw, fd)
#             print 'correct:'
#             print sess.run(correct, fd)
#             print 'total_seqlen:'
#             print sess.run(total_seqlen, fd)
#             print 'accuracy:'
#             print sess.run(accuracy, fd)
#             assert 1==0
            
            for i, (inp, pred, tar, length) in enumerate(zip(fd[encoder_inputs].T, predict_.T, fd[decoder_targets].T, lengths_)):
                print('  sample {}:'.format(i + 1))
                decoded_inp = decode_seq(inp)
                print('    input     > {}'.format(decoded_inp))
                print('    predicted > {}'.format(decode_by_index(decoded_inp, pred, end_idx=length)))
                print('    target    > {}'.format(decode_by_index(decoded_inp, tar, end_idx=length)))
#                 print('    predicted > {}'.format(decode_order(pred))) # these print out raw indices
#                 print('    target    > {}'.format(decode_order(tar)))
                if i >= 2:
                    break
            print
            
    # EVALUATE ON A BIG TEST SET
    loss_track, accuracy_track = [], []
    for _ in range(num_test_batches):
        fd = next_feed(BATCH_SIZE)
        l, a = sess.run([loss, accuracy], fd)
        loss_track.append(l)
        accuracy_track.append(a)
    print('Evaluation results (on {} batches):'.format(num_test_batches))
    print('  average loss: {} | average accuracy {}'.format(np.mean(loss_track), np.mean(accuracy_track)))
    print
    
    
except KeyboardInterrupt:
    print('training interrupted')

Batch 0
  minibatch loss: 0.16543418169 | accuracy 0.846153855324
  sample 1:
    input     > ['6', '2', '8', '9', '1', '0', '3', '5', '7']
    predicted > ['1', '2', '0', '5', '7', '6', '7', '8', '9']
    target    > ['0', '1', '2', '3', '5', '6', '7', '8', '9']
  sample 2:
    input     > ['4', '1', '9', '3', '8', '5', '7', 'PAD', 'PAD']
    predicted > ['1', '3', '4', '5', '7', '8', '9']
    target    > ['1', '3', '4', '5', '7', '8', '9']
  sample 3:
    input     > ['5', '9', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
    predicted > ['5', '9']
    target    > ['5', '9']

Batch 1000
  minibatch loss: 0.144738689065 | accuracy 0.905660390854
  sample 1:
    input     > ['6', '4', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
    predicted > ['4', '6']
    target    > ['4', '6']
  sample 2:
    input     > ['3', '2', '6', '9', 'PAD', 'PAD', 'PAD', 'PAD']
    predicted > ['2', '3', '6', '9']
    target    > ['2', '3', '6', '9']
  sample 3:
    input     > ['8', '2', '7', '4', '5', '

In [216]:
a,b = random_datum(5)
print type(a)
print type(b)
print a,b

<type 'list'>
<type 'list'>
['3', '9', '7', '5', '1'] [6, 2, 5, 4, 3]


In [221]:
decode_by_index(a,b,5)

['1', '3', '5', '7', '9']

In [219]:
for b_ in b:
    print a[b_-2], 

1 3 5 7 9


In [218]:
np.array(a)

array(['3', '9', '7', '5', '1'], 
      dtype='|S1')

In [205]:
next_feed(5)

{<tf.Tensor 'encoder_inputs:0' shape=(?, ?) dtype=int32>: array([[10,  9,  4, 11, 10],
        [ 2,  7,  6,  7,  6],
        [ 4,  2,  5,  6,  2],
        [ 5, 10, 11,  5,  8],
        [ 0,  3,  2,  8,  0]], dtype=int32),
 <tf.Tensor 'encoder_inputs_length:0' shape=(?,) dtype=int32>: [4, 5, 5, 5, 4],
 <tf.Tensor 'decoder_targets:0' shape=(?, ?) dtype=int32>: array([[3, 4, 5, 2, 4],
        [4, 6, 6, 5, 3],
        [5, 3, 2, 4, 5],
        [2, 2, 4, 3, 2],
        [1, 5, 3, 6, 1],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=int32)}

In [12]:
a, b = random_batch(5)
print a
print b

[[3, 9, 8], [9, 6, 8], [10, 2, 6, 8, 7, 3, 9, 11, 5], [9, 8, 11, 10, 5, 4, 6], [6, 8, 4, 11, 2, 9, 5, 10, 3]]
[[0, 2, 1], [1, 2, 0], [7, 1, 5, 8, 2, 4, 3, 6, 0], [2, 5, 4, 6, 1, 0, 3], [3, 4, 8, 2, 6, 0, 1, 5, 7]]


In [15]:
utils.batch([seq + [WORD2IDX['EOS']] + [WORD2IDX['PAD']]*2 for seq in b], custom_pad=MAX_LEN-1)

(array([[ 0,  1,  7,  2,  3],
        [ 2,  2,  1,  5,  4],
        [ 1,  0,  5,  4,  8],
        [ 1,  1,  8,  6,  2],
        [ 0,  0,  2,  1,  6],
        [ 0,  0,  4,  0,  0],
        [11, 11,  3,  3,  1],
        [11, 11,  6,  1,  5],
        [11, 11,  0,  0,  7],
        [11, 11,  1,  0,  1],
        [11, 11,  0, 11,  0],
        [11, 11,  0, 11,  0]], dtype=int32), [6, 6, 12, 10, 12])

In [16]:
utils.batch(b, custom_pad=MAX_LEN-1)

(array([[ 0,  1,  7,  2,  3],
        [ 2,  2,  1,  5,  4],
        [ 1,  0,  5,  4,  8],
        [11, 11,  8,  6,  2],
        [11, 11,  2,  1,  6],
        [11, 11,  4,  0,  0],
        [11, 11,  3,  3,  1],
        [11, 11,  6, 11,  5],
        [11, 11,  0, 11,  7]], dtype=int32), [3, 3, 9, 7, 9])

In [ ]:

def random_batch(batch_size, input_length_from=2, input_length_to=MAX_LEN-2):
    if input_length_from >= input_length_to:
        raise ValueError('length_from >= length_to')
    input_lengths = np.random.randint(input_length_from, input_length_to, size=batch_size)
    input_batch, output_batch = [], []
    for length in input_lengths:
        input_seq, output_seq = random_datum(length)
        input_batch.append(encode_seq(input_seq))
        output_batch.append(output_seq)
    return input_batch, output_batch    
